### Spherical K-means as Efficiency Improvement for Vector Space Model

Spherical K-means is a widely used adaptation of the K-means algorithm, modified to cluster sparse high-dimensional data objects (i.e. text data). This code was written as a practical application of ideas covered by various academics over recent decades. Spherical K-means is used here as an extension to a vector space model search engine, with the goal of boosting search efficiency. By narrowing the search scope to the most relevant cluster (i.e. topic cluster). It reduces the number of document comparisons needed to find relevant articles, potentially improving both speed and relevance for large datasets. The dataset used is a reduced version of a json file holding roughly 19000 wikipedia articles (text, ids, titles etc.).

This code was used on the dataset with roughly 1900 articles, could easily be expanded.


### KMeans Tweak For Cosine Similarity:

KMeans clustering with normalisation is used so that the euclidean distance calculation is more akin to cosine similarity. This normalisation step ensures that the document vectors are on a unit sphere, making the Euclidean distance between them more reflective of the angle between vectors. The clustering now indirectly attempts to minimise angular differences rather than euclidean distances. The initial clustering process now also focusses on the direction (semantics) rather than magnitude (length or frequency). 

#### Loading and Preprocessing:

Loading: This function loads the text data from a compressed (zip) file containing JSON objects. Each JSON object represents a Wikipedia article.

Preprocessing: While this code snippet primarily focuses on loading the articles, in a full implementation, preprocessing would typically involve cleaning the text (e.g., removing punctuation, lowercasing, etc.) and possibly tokenizing or lemmatizing the text.

In [24]:
import json
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def load_and_preprocess(file_path):
    articles_text = []
    articles_titles = []  # Store article titles
    with zipfile.ZipFile(file_path, 'r') as z:
        with z.open(z.namelist()[0]) as f:
            for line in f:
                article = json.loads(line)
                # Combine the title and text
                combined_text = article['title'] + " " + article['text']
                articles_text.append(combined_text)  # Use combined text for TF-IDF
                articles_titles.append(article['title'])  # Keep titles for later retrieval
    return articles_text, articles_titles


articles_text, articles_titles = load_and_preprocess('/Users/andrew/Documents/Programming For AI&Data/wiki-articles-small.json.zip')


#### Vectorizing the Text with TF-IDF

TF-IDF Vectorization: Converts the preprocessed text documents into a matrix of TF-IDF features. It transforms the text into a numerical representation that reflects how important a word is to a document in the context of the entire corpus. Stop words (commonly used words of little value) are removed to improve the analysis.

In [25]:

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_text)

#### Normalize the TF-IDF Vectors

Normalization: Adjusts the TF-IDF vectors so that each vector has a magnitude (or length) of 1. This is done by scaling the vector components such that the vector lies on a unit sphere. This step ensures that the distance calculations in the subsequent clustering phase are more reflective of the angle between vectors, aligning better with the concept of cosine similarity.

In [26]:

normalized_vectors = normalize(tfidf_matrix)


#### Cluster the Documents Using K-means

Clustering: Applies the K-means algorithm to the normalized TF-IDF vectors to group documents into clusters based on their similarities. 

In [27]:

kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans.fit(normalized_vectors)
clusters = kmeans.labels_


#### Creates an Index Mapping Clusters to Articles

Index Creation: Constructs a mapping from clusters to document indices, facilitating the retrieval of documents belonging to a specific cluster. This index is crucial for efficiently finding relevant documents during the search process.

In [28]:

def create_cluster_index(clusters):
    cluster_index = {}
    for i, cluster in enumerate(clusters):
        if cluster not in cluster_index:
            cluster_index[cluster] = []
        cluster_index[cluster].append(i)
    return cluster_index

cluster_index = create_cluster_index(clusters)

#### Process a Search Query

Implements a search function that identifies the most relevant cluster to a query, calculates the cosine similarity within that cluster, and returns the titles of the top N most similar articles.

In [29]:
# Define the search function
def search(query, tfidf_vectorizer, kmeans_model, normalized_vectors, cluster_index, articles_titles, top_n=5):
    
    query = query.lower() # Note, the tf-idf vectorizer converts tokens to lowercase by default
    
    # Vectorize and normalize query
    query_vector = tfidf_vectorizer.transform([query])
    normalized_query_vector = normalize(query_vector)
    
    # Identify the most relevant cluster
    cluster_centroids = kmeans_model.cluster_centers_
    query_cluster_similarities = cosine_similarity(normalized_query_vector, cluster_centroids)
    most_relevant_cluster = query_cluster_similarities.argmax()
    
    # Find the most similar documents within the cluster
    docs_in_relevant_cluster = cluster_index[most_relevant_cluster]
    doc_similarities = cosine_similarity(normalized_query_vector, normalized_vectors[docs_in_relevant_cluster]).flatten()
    most_similar_docs_indices = np.argsort(doc_similarities)[-top_n:][::-1]
    
    # Returns most similar titles, can be altered to return text instead
    most_similar_titles = [articles_titles[docs_in_relevant_cluster[idx]] for idx in most_similar_docs_indices]
    return most_similar_titles


In [30]:
# Example usage
query = "Python Programming"
top_titles = search(query, tfidf_vectorizer, kmeans, normalized_vectors, cluster_index, articles_titles, top_n=10)

for title in top_titles:
    print(title)


Timeline of programming languages
Lynx (programming language)
Programmer
Niklaus Wirth
Design Patterns
Recursion
Software
Prolog
Assembly language
Self-reference


#### Full Code:

In [31]:
import json
import zipfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Step 1: Load articles and preprocess
def load_and_preprocess(file_path):
    articles_text = []
    articles_titles = []  # Store article titles
    with zipfile.ZipFile(file_path, 'r') as z:
        with z.open(z.namelist()[0]) as f:
            for line in f:
                article = json.loads(line)
                # Combine the title and text
                combined_text = article['title'] + " " + article['text']
                articles_text.append(combined_text)  # Use combined text for TF-IDF
                articles_titles.append(article['title'])  # Keep titles for later retrieval
    return articles_text, articles_titles


articles_text, articles_titles = load_and_preprocess('/Users/andrew/Documents/Programming For AI&Data/wiki-articles-small.json.zip')

# Step 2: Vectorize the combined text and titles using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(articles_text)

# Normalize the TF-IDF vectors
normalized_vectors = normalize(tfidf_matrix)

# Step 3: Cluster the documents using K-means
kmeans = KMeans(n_clusters=5, random_state=42, n_init=10)
kmeans.fit(normalized_vectors)
clusters = kmeans.labels_

# Create an index mapping clusters to articles
def create_cluster_index(clusters):
    cluster_index = {}
    for i, cluster in enumerate(clusters):
        if cluster not in cluster_index:
            cluster_index[cluster] = []
        cluster_index[cluster].append(i)
    return cluster_index

cluster_index = create_cluster_index(clusters)

# Step 4: Define the search function
def search(query, tfidf_vectorizer, kmeans_model, normalized_vectors, cluster_index, articles_titles, top_n=5):
    
    query = query.lower()
    
    # Vectorize and normalize query
    query_vector = tfidf_vectorizer.transform([query])
    normalized_query_vector = normalize(query_vector)
    
    # Identify the most relevant cluster
    cluster_centroids = kmeans_model.cluster_centers_
    query_cluster_similarities = cosine_similarity(normalized_query_vector, cluster_centroids)
    most_relevant_cluster = query_cluster_similarities.argmax()
    
    # Find the most similar documents within the cluster
    docs_in_relevant_cluster = cluster_index[most_relevant_cluster]
    doc_similarities = cosine_similarity(normalized_query_vector, normalized_vectors[docs_in_relevant_cluster]).flatten()
    most_similar_docs_indices = np.argsort(doc_similarities)[-top_n:][::-1]
    
    # Returns most similar titles, can be altered to return text instead
    most_similar_titles = [articles_titles[docs_in_relevant_cluster[idx]] for idx in most_similar_docs_indices]
    return most_similar_titles

# Example usage
query = "Python Programming"
top_titles = search(query, tfidf_vectorizer, kmeans, normalized_vectors, cluster_index, articles_titles, top_n=10)

for title in top_titles:
    print(title)


Timeline of programming languages
Lynx (programming language)
Programmer
Niklaus Wirth
Design Patterns
Recursion
Software
Prolog
Assembly language
Self-reference


References:

Hornik, K., Feinerer, I., Kober, M. and Buchta, C., 2012. Spherical k-means clustering. Journal of statistical software, 50, pp.1-22.

Huang, A., 2008, April. Similarity measures for text document clustering. In Proceedings of the sixth new zealand computer science research student conference (NZCSRSC2008), Christchurch, New Zealand (Vol. 4, pp. 9-56).

Zhong, S., 2005, July. Efficient online spherical k-means clustering. In Proceedings. 2005 IEEE International Joint Conference on Neural Networks, 2005. (Vol. 5, pp. 3180-3185). IEEE.

-----

Important quotes for choice of K-means: 

Huang:

"Partitional clustering algorithms have been recognized to be better suited for handling large document datasets than hi- erarchical ones, due to their relatively low computational requirements [16, 9, 3]." p.52
